In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# 加载LLaMA-3模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
# llama_model = AutoModelForCausalLM.from_pretrained("bofenghuang/Meta-Llama-3-8B", device_map="auto", torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map="auto")
model.eval()

/mnt/miniconda3/envs/anomaly_clip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/miniconda3/envs/anomaly_clip/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


cuda


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-66bc0ee4-33b0444e256fd9a812c15435;0ed20ad6-62c4-4411-b9de-c89fb0881b7c)

Cannot access gated repo for url https://hf-mirror.com/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must be authenticated to access it.

In [6]:
def generate_conversation(model, tokenizer, prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            max_length=max_length
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

dialogue_prompt = "Person A: Hi there! How are you doing today?\nPerson B:"
generated_conversation = generate_conversation(model, tokenizer, dialogue_prompt)
print("Generated Conversation:", generated_conversation)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Conversation: Person A: Hi there! How are you doing today?
Person B: I’m doing well, thanks for asking! How are you?
Person A: I’m good, thanks! How’s your day going?
Person B: Pretty good. I’m just working on some projects right now.
Person A: That’s great! What kind of projects are you working on?
Person B: I’m working on a few different things. I’m writing a blog post, working on a design project


In [ ]:
# Calculate overall accuracy
total_images = 0
correct_predictions = 0
similarity_threshold = 0.95

for object_type in meta['train']:
    results_file = f"{results_dir}zs_1_NS_{object_type}_test_results.txt"
    
    if not os.path.exists(results_file):
        continue

    with open(results_file, 'r') as file:
        lines = file.readlines()
        total_images += len(lines)
        for line in lines:
            if 'is predicted as' in line:
                predicted_text = line.split('is predicted as')[-1].strip()
                predicted_embedding = get_text_embedding(predicted_text)
                
                image_path = line.split()[1]
                true_label = next((item['specie_name'] for item in meta['test'][object_type] if item['img_path'] in image_path), None)

                if true_label:
                    true_embedding = get_text_embedding(true_label)
                    true_similarity = cosine_similarity(predicted_embedding, true_embedding).flatten()[0]

                    if true_similarity >= similarity_threshold:
                        correct_predictions += 1
                else:
                    print(f"True label not found for image path: {image_path}")

accuracy = correct_predictions / total_images if total_images > 0 else 0
print(f"Overall Accuracy: {accuracy:.2f}")

# Save statistics to file
stats_file = os.path.join(results_dir, "prediction_stats.txt")
with open(stats_file, 'w') as file:
    file.write(f"Total Images: {total_images}\n")
    file.write(f"Correct Predictions: {correct_predictions}\n")

print(f"Prediction statistics saved to {stats_file}")

NameError: name 'meta' is not defined